In [34]:
from langchain.chains import RetrievalQA
from langchain_ollama import ChatOllama
from langchain_ollama.embeddings import OllamaEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.document_loaders import CSVLoader
from langchain.indexes import VectorstoreIndexCreator

In [35]:
file = "OutdoorClothingCatalog_1000.csv"
loader = CSVLoader(file)
data = loader.load()

In [36]:
embeddings = OllamaEmbeddings(model="mxbai-embed-large")

In [37]:
# from langchain_community.embeddings import OllamaEmbeddings

# embeddings = OllamaEmbeddings(model="mxbai-embed-large")

In [38]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embeddings
).from_loaders([loader])

In [39]:
llm = ChatOllama(model="llama3.2:latest", temperature=0)
qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=index.vectorstore.as_retriever(), 
    verbose=True,
    chain_type_kwargs = {
        "document_separator": "<<<<>>>>>"
    }
)

In [40]:
data[10]

Document(metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 10}, page_content=": 10\nname: Cozy Comfort Pullover Set, Stripe\ndescription: Perfect for lounging, this striped knit set lives up to its name. We used ultrasoft fabric and an easy design that's as comfortable at bedtime as it is when we have to make a quick run out.\n\nSize & Fit\n- Pants are Favorite Fit: Sits lower on the waist.\n- Relaxed Fit: Our most generous fit sits farthest from the body.\n\nFabric & Care\n- In the softest blend of 63% polyester, 35% rayon and 2% spandex.\n\nAdditional Features\n- Relaxed fit top with raglan sleeves and rounded hem.\n- Pull-on pants have a wide elastic waistband and drawstring, side pockets and a modern slim leg.\n\nImported.")

In [41]:
data[11]

Document(metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 11}, page_content=': 11\nname: Ultra-Lofty 850 Stretch Down Hooded Jacket\ndescription: This technical stretch down jacket from our DownTek collection is sure to keep you warm and comfortable with its full-stretch construction providing exceptional range of motion. With a slightly fitted style that falls at the hip and best with a midweight layer, this jacket is suitable for light activity up to 20° and moderate activity up to -30°. The soft and durable 100% polyester shell offers complete windproof protection and is insulated with warm, lofty goose down. Other features include welded baffles for a no-stitch construction and excellent stretch, an adjustable hood, an interior media port and mesh stash pocket and a hem drawcord. Machine wash and dry. Imported.')

Hardcoded examples

In [42]:
examples = [
    {
        "query": "Do the Cozy Comfort Pullover Set\
        have side pockets?",
        "answer": "Yes"
    },
    {
        "query": "What collection is the Ultra-Lofty \
        850 Stretch Down Hooded Jacket from?",
        "answer": "The DownTek collection"
    }
]

LLM generated

In [43]:
from langchain.evaluation.qa import QAGenerateChain

In [44]:
example_gen_chain = QAGenerateChain.from_llm(ChatOllama(model="llama3.2:latest", temperature=0))

In [45]:
new_examples = example_gen_chain.apply_and_parse(
    [{"doc": t} for t in data[:5]]
)

/Users/ismasadou/Documents/kuude/fulfulde-translator/.venv/lib/python3.13/site-packages/langchain/chains/llm.py:369: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


In [46]:
new_examples[0]

{'qa_pairs': {'query': "What is the recommended method for ordering shoe sizes for the Women's Campside Oxfords?",
  'answer': 'To order regular shoe size, and for half sizes not offered, to order up to the next whole size.'}}

In [47]:
data[0]

Document(metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 0}, page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries.")

In [50]:
# examples += new_examples
new_examples_clean = [{"query": e["qa_pairs"]["query"], "answer": e["qa_pairs"]["answer"]} for e in new_examples]
print(new_examples_clean)

[{'query': "What is the recommended method for ordering shoe sizes for the Women's Campside Oxfords?", 'answer': 'To order regular shoe size, and for half sizes not offered, to order up to the next whole size.'}, {'query': 'What percentage of recycled materials are used in the construction of the Recycled Waterhog Dog Mat?', 'answer': '94%'}, {'query': "What is the UPF rating of the fabric in the Infant and Toddler Girls' Coastal Chill Swimsuit, Two-Piece?", 'answer': 'UPF 50+'}, {'query': 'What percentage of recycled nylon is used in the body of the Refresh Swimwear tankini top?', 'answer': '82%'}, {'query': 'What is the primary benefit of the TEK O2 technology in the EcoFlex 3L Storm Pants?', 'answer': 'The primary benefit of the TEK O2 technology is enhanced breathability, allowing for better airflow and keeping the wearer dry and comfortable.'}]


In [51]:
examples += new_examples_clean
print(examples)

[{'query': 'Do the Cozy Comfort Pullover Set        have side pockets?', 'answer': 'Yes'}, {'query': 'What collection is the Ultra-Lofty         850 Stretch Down Hooded Jacket from?', 'answer': 'The DownTek collection'}, {'query': "What is the recommended method for ordering shoe sizes for the Women's Campside Oxfords?", 'answer': 'To order regular shoe size, and for half sizes not offered, to order up to the next whole size.'}, {'query': 'What percentage of recycled materials are used in the construction of the Recycled Waterhog Dog Mat?', 'answer': '94%'}, {'query': "What is the UPF rating of the fabric in the Infant and Toddler Girls' Coastal Chill Swimsuit, Two-Piece?", 'answer': 'UPF 50+'}, {'query': 'What percentage of recycled nylon is used in the body of the Refresh Swimwear tankini top?', 'answer': '82%'}, {'query': 'What is the primary benefit of the TEK O2 technology in the EcoFlex 3L Storm Pants?', 'answer': 'The primary benefit of the TEK O2 technology is enhanced breathab

In [52]:
qa.run(examples[0]["query"])



> Entering new RetrievalQA chain...

> Finished chain.


"I don't know, I couldn't find that information in the provided context."

Manual evaluation

In [53]:
import langchain
langchain.debug = True

In [54]:
qa.run(examples[0]["query"])

[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "Do the Cozy Comfort Pullover Set        have side pockets?"
}
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "question": "Do the Cozy Comfort Pullover Set        have side pockets?",
  "context": ": 10\nname: Cozy Comfort Pullover Set, Stripe\ndescription: Perfect for lounging, this striped knit set lives up to its name. We used ultrasoft fabric and an easy design that's as comfortable at bedtime as it is when we have to make a quick run out.\n\nSize & Fit\n- Pants are Favorite Fit: Sits lower on the waist.\n- Relaxed Fit: Our most generous fit sits farthest from the body.\n\nFabric & Care\n- In the softest blend of 63% polyester, 35% rayon and 2% spandex.\n\nAdditional Features\n- Relaxed fit top with raglan sleeves and rounded hem.\n- Pull-

"I don't know, I couldn't find that information in the provided context."

In [55]:
# Turn off the debug mode
langchain.debug = False

In [56]:
print(examples)

[{'query': 'Do the Cozy Comfort Pullover Set        have side pockets?', 'answer': 'Yes'}, {'query': 'What collection is the Ultra-Lofty         850 Stretch Down Hooded Jacket from?', 'answer': 'The DownTek collection'}, {'query': "What is the recommended method for ordering shoe sizes for the Women's Campside Oxfords?", 'answer': 'To order regular shoe size, and for half sizes not offered, to order up to the next whole size.'}, {'query': 'What percentage of recycled materials are used in the construction of the Recycled Waterhog Dog Mat?', 'answer': '94%'}, {'query': "What is the UPF rating of the fabric in the Infant and Toddler Girls' Coastal Chill Swimsuit, Two-Piece?", 'answer': 'UPF 50+'}, {'query': 'What percentage of recycled nylon is used in the body of the Refresh Swimwear tankini top?', 'answer': '82%'}, {'query': 'What is the primary benefit of the TEK O2 technology in the EcoFlex 3L Storm Pants?', 'answer': 'The primary benefit of the TEK O2 technology is enhanced breathab

LLM assisted

In [62]:
# predictions = qa.apply(examples[["query"]])
# predictions = [qa.run(ex["query"]) for ex in examples]
predictions = [
    {
        "query": ex["query"],
        "result": qa.run(ex["query"]),
        "answer": ex["answer"]
    } for ex in examples
]



> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


In [63]:
from langchain.evaluation.qa import QAEvalChain

In [64]:
eval_chain = QAEvalChain.from_llm(llm)

In [65]:
print(predictions[0])

{'query': 'Do the Cozy Comfort Pullover Set        have side pockets?', 'result': "I don't know, I couldn't find that information in the provided context.", 'answer': 'Yes'}


In [66]:
graded_outputs = eval_chain.evaluate(examples, predictions)

In [68]:
graded_outputs[0]

{'results': 'INCORRECT\n\nThe student answer is not providing a factual answer to the question. Instead of stating whether or not the Cozy Comfort Pullover Set has side pockets, they are expressing uncertainty and lack of knowledge about the topic. A correct answer would have simply stated "Yes" or "No".'}

In [69]:
for i, eg in enumerate(examples):
    print(f"Example {i}:")
    print("Question: " + predictions[i]['query'])
    print("Real Answer: " + predictions[i]['answer'])
    print("Predicted Answer: " + predictions[i]['result'])
    print("Predicted Grade: " + graded_outputs[i]['results'])
    print()

Example 0:
Question: Do the Cozy Comfort Pullover Set        have side pockets?
Real Answer: Yes
Predicted Answer: I don't know, I couldn't find that information in the provided context.
Predicted Grade: INCORRECT

The student answer is not providing a factual answer to the question. Instead of stating whether or not the Cozy Comfort Pullover Set has side pockets, they are expressing uncertainty and lack of knowledge about the topic. A correct answer would have simply stated "Yes" or "No".

Example 1:
Question: What collection is the Ultra-Lofty         850 Stretch Down Hooded Jacket from?
Real Answer: The DownTek collection
Predicted Answer: The Ultra-Lofty 850 Stretch Down Hooded Jacket comes from the DownTek collection.
Predicted Grade: GRADE: CORRECT

Example 2:
Question: What is the recommended method for ordering shoe sizes for the Women's Campside Oxfords?
Real Answer: To order regular shoe size, and for half sizes not offered, to order up to the next whole size.
Predicted Answe